In [ ]:
import torch
import pandas as pd
import time
import json
import flask, json
from flask import request
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import (
    AutoConfig,
    AutoModelForQuestionAnswering,
    AutoTokenizer,
    squad_convert_examples_to_features
)
from transformers.data.processors.squad import SquadResult, SquadV2Processor, SquadExample
from transformers.data.metrics.squad_metrics import compute_predictions_logits
#起服务
server = flask.Flask(__name__)
#加载模型
model = AutoModelForQuestionAnswering.from_pretrained('/home/wangwuyue/cuad-models/deberta-v2-xlarge/')
tokenizer = AutoTokenizer.from_pretrained('/home/wangwuyue/cuad-models/deberta-v2-xlarge/', use_fast=True)
#加载数据
with open('/home/wangwuyue/cuad-data/CUAD_v1/CUAD_v1.json') as json_file:
    data = json.load(json_file)
#构建prediction函数，传入问题，待推理文本，模型路径
def run_prediction(question_texts, context_text, model_path):
    ### Setting hyperparameters
    max_seq_length = 512
    doc_stride = 256
    n_best_size = 1
    max_query_length = 64
    max_answer_length = 512
    do_lower_case = False
    null_score_diff_threshold = 0.0

    # model_name_or_path = "../cuad-models/roberta-base/"

    def to_list(tensor):
        return tensor.detach().cpu().tolist()

    config_class, model_class, tokenizer_class = (
        AutoConfig, AutoModelForQuestionAnswering, AutoTokenizer)
    config = config_class.from_pretrained(model_path)
    tokenizer = tokenizer_class.from_pretrained(
        model_path, do_lower_case=True, use_fast=False)
    model = model_class.from_pretrained(model_path, config=config)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    processor = SquadV2Processor()
    examples = []

    for i, question_text in enumerate(question_texts):
        example = SquadExample(
            qas_id=str(i),
            question_text=question_text,
            context_text=context_text,
            answer_text=None,
            start_position_character=None,
            title="Predict",
            answers=None,
        )

        examples.append(example)

    features, dataset = squad_convert_examples_to_features(
        examples=examples,
        tokenizer=tokenizer,
        max_seq_length=max_seq_length,
        doc_stride=doc_stride,
        max_query_length=max_query_length,
        is_training=False,
        return_dataset="pt",
        threads=1,
    )

    eval_sampler = SequentialSampler(dataset)
    eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=10)

    all_results = []

    for batch in eval_dataloader:
        model.eval()
        batch = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "token_type_ids": batch[2],
            }

            example_indices = batch[3]

            outputs = model(**inputs)

            for i, example_index in enumerate(example_indices):
                eval_feature = features[example_index.item()]
                unique_id = int(eval_feature.unique_id)

                output = [to_list(output[i]) for output in outputs.to_tuple()]

                start_logits, end_logits = output
                result = SquadResult(unique_id, start_logits, end_logits)
                all_results.append(result)

    final_predictions = compute_predictions_logits(
        all_examples=examples,
        all_features=features,
        all_results=all_results,
        n_best_size=n_best_size,
        max_answer_length=max_answer_length,
        do_lower_case=do_lower_case,
        output_prediction_file=None,
        output_nbest_file=None,
        output_null_log_odds_file=None,
        verbose_logging=False,
        version_2_with_negative=True,
        null_score_diff_threshold=null_score_diff_threshold,
        tokenizer=tokenizer
    )

    return final_predictions
#获得干净的问题清单（41类）
questions = []
for i, q in enumerate(data['data'][0]['paragraphs'][0]['qas']):
    question = data['data'][0]['paragraphs'][0]['qas'][i]['question']
    questions.append(question)
for i in range(len(questions)):
    questions[i]=questions[i].replace('\xa0','')
    
#起服务
@server.route('/ContractExtractionEN', methods=['get', 'post'])

def get_result():
    results = []
    try:
        start = time.process_time()
        #my_ie = Taskflow("information_extraction", schema=schema, task_path=r'/root/py220725/model_700-20220621T013625Z-001/model_700')
        contract = request.values.get('contract')
        contract=contract.replace('\n','')
        contract=contract.replace('\xa0','')
        contract=contract.replace('\t','')
        predictions = []
        #results = return_result(companyName)
        start = time.perf_counter()
        predictions = run_prediction(questions, contract,'/home/wangwuyue/cuad-models/deberta-v2-xlarge')
        results={}
        for i in range(len(questions)):
            ques = questions[i]
            results[ques] = list(predictions.items())[i][1]
        end = time.perf_counter()
        print(end-start)
    except Exception as e:
        print(e)
        return json.dumps({'code':'400003','message':'失败','results': results}, ensure_ascii=False)  # 将字典转换为json串, json是字符串
    if len(results)==0:
        return json.dumps({'code':'400002','message':'成功但是没有结果','results': results}, ensure_ascii=False)
    else:
        return json.dumps({'code':'400001','message':'成功','results': results}, ensure_ascii=False)  # 将字典转换为json串, json是字符串
    #print('')

if __name__ == '__main__':
    server.run(debug=False, port=8906, host='0.0.0.0') # 指定端口、host,0.0.0.0代表不管几个网卡，任何ip都可以访问

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8905
 * Running on http://172.17.16.159:8905
Press CTRL+C to quit
add example index and unique id: 100%|██████████| 41/41 [00:00<00:00, 174940.45it/s]
10.116.84.2 - - [24/Feb/2023 21:35:17] "GET /ContractExtractionEN?contract=Exhibit%2010.13Confidential%20Materials%20omitted%20and%20filed%20separately%20with%20the%20Securities%20and%20Exchange%20Commission.%20Asterisks%20denote%20omissions.DATED:%20OCTOBER%2015,%202009PACIRA%20PHARMACEUTICALS,%20INC.andEKR%20THERAPEUTICS,%20INC.%20%20%20%20%20%20%20%20AMENDED%20AND%20RESTATED%20%20%20STRATEGIC%20LICENSING,%20DISTRIBUTION%20AND%20MARKETING%20AGREEMENTTHIS%20AMENDED%20AND%20RESTATED%20STRATEGIC%20LICENSING,%20DISTRIBUTION%20AND%20MARKETING%20AGREEMENT%20(the%20%20%22Agreement%22)%20is%20made%20on%20October%2015,%202009%20(the%20%22Agreement%20Date%22)%20and%20is%20effective%20as%20of%20the%20Effective%20Date%20(as%20defined%20below),%20between:%20%20%20PACIRA%20PHARMACE

14.880687652155757


10.116.84.2 - - [24/Feb/2023 21:35:17] "GET /favicon.ico HTTP/1.1" 404 -


In [ ]:
pip install git+https://github.com/PaddlePaddle/PaddleNLP.git

from paddlenlp.transformers import RobertaForQuestionAnswering, RobertaBPETokenizer
tokenizer = RobertaBPETokenizer.from_pretrained("TracyWang/MAUD_KWM_AWS_Roberta-base", from_hf_hub=True)
model = RobertaForQuestionAnswering.from_pretrained("TracyWang/MAUD_KWM_AWS_Roberta-base", from_hf_hub=True)
input_features = tokenizer(question, context, return_tensors="pd")
start_logits, end_logits = model(**input_features)